# Dropping Boxes

Question:

*You have two identical boxes, and each box will break if dropped from the $N$th floor (or higher) of a 100-story building. What is your strategy for guaranteeing finding $N$ in the minimum number of drops? Given your strategy, what is the worst case number of drops?*

Let's begin thinking about possible strategies

**2 floor stategy** is based on sweeping the building upwards in a way that guarantees you find the $N$th floor.

- You start at the 2nd floor and drop one box. 
    - If it breaks, you know that $N=2$ or $N=1$. So you go to the 1st floor and drop the second box. If it breaks you know $N=1$, if if does not break then $N=2$. **Solved**
    - If it does not break, you automatically know that all floors below and including the 2nd floor are safe. So you can regard the 2nd floor as your 'new ground floor'. You go 2 floors up to the 4th floor and repeat the process. If the first box breaks, it will be either $N=4$ or $N=3$, and you can use the second box to find out.

This strategy of going up every 2 floors requires a minimum of 1 drop to clear 2 floors (if the first box doesn't break) and a maximum of 2 drops to find $N$ for that set of floors: one for the first box to break and another one to check the floor below

**3 floor strategy**, could you sweep every 3 floors and still find $N$?

If you went up every 3 floors or more... Let's imagine you start at the 3rd floor and the box breaks. You have only one box left to check whether $N=3$, $N=2$ or $N=1$ so to be entirely sure you have to go to the 1st floor, drop the second box; if it doesn't break you drop it from the 2nd floor and if it doesn't break you know it was $N=3$. So with 1 drop you can potentially clear 3 floors and in the worst case scenario $N=3$ you find it in 3 drops.

This strategy is better because it discards floors quicker (if you are lucky) and it requires 3 drops to check if $N=3$ while the previous strategy requires an equivalent number of 2 drops to check if $N=2$.

This can be generalized to an **$f$ floor strategy** where you start by dropping the first box from floor $f$ and if it breaks, you go up, floor by floor until $f-1$.

### How many drops?

How many drops would you need to find the $N$th floor?
If we are following an **$f$ floor strategy**, there can be two outcomes:
- If $N < f$ we will need 1 drop for the first box to break and another $N$ drops to find the $N$ floor
    - so the total number of drops will be $d = N + 1$. 
- If $N > f$ you will need $floor(N / f) + 1$ drops of the first box until it breaks and then sweep the floors upwards starting from $floor(N / f) \cdot f$ until $N$
    - so the total number of drops will be $d = floor(N / f) + 1 + (N - floor(N / f) \cdot f)$

We can use this to calculate how many drops will be needed to find the $N$th floor for a given $f$ strategy, for each case of $N$. The code below calculates that:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def drops(N, f):
    if N < f:
        return N + 1
    if N == f:
        return N
    if N > f:
        return int(np.floor(N / f) + 1 + (N - np.floor(N / f) * f))
    
N_floors = np.arange(1, 100 + 1)

For example, if $N=5$ and we follow a $f=2$ strategy, we would need 4 drops to find $N$. 
- (1) We drop the first box from the 2nd floor. It doesn't break
- (2) We drop the first box from the 4rd floor. It doesn't break
- (3) We drop the first box from the 6th floor. It breaks
- (4) We drop the second box from the 5th floor. It breaks. $N$= 5


In [ ]:
drops(N=5, f=2)

### Comparing strategies

We can compare different $f$ strategies if we calculate the number of drops needed to find the $N$th floor for all possible cases. The plot below shows the results for $f=2, 3, 4, 5$

In [ ]:
drops_f2 = [drops(N=n, f=2) for n in N_floors]

drops_f3 = [drops(N=n, f=3) for n in N_floors]

drops_f4 = [drops(N=n, f=4) for n in N_floors]

drops_f5 = [drops(N=n, f=5) for n in N_floors]


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
ax.scatter(N_floors, drops_f2, s=10, label='2')
ax.scatter(N_floors, drops_f3, s=10, label='3')
ax.scatter(N_floors, drops_f4, s=10, label='4')
ax.scatter(N_floors, drops_f5, s=10, label='5')
ax.legend(title='f floors')
ax.set_xlabel(r'N-th floor')
ax.set_ylabel(r'Drops needed')
plt.show()

Here we might be tempted to think that increasing $f$ always leads to an improvement in the maximum number of drops, but we have to be very careful. Skipping too many floors becomes inefficient. For example, if we start by dropping the box from the 50th floor and $N=50$, we would need to try every single floor up to the 49th

### Maximum number of drops

If we look at the maximum number of drops needed to find the $N$th floor as a function of the $f$ strategy
we notice that the pattern goes as $D_{max} = floor(M / f) + (f - 1)$ where $M$ is the total number of floors

This has two opposing terms: if we use a small $f$, the first term shows us that we go up the building very slowly (as the plot above showed) so the total number of drops will be large. But if we go up too fast because $f$ is large, we are also very inefficient

In [ ]:
print("Maximum number of drops for f=2 : %d" % np.max(drops_f2))
print("Maximum number of drops for f=3 : %d" % np.max(drops_f3))
print("Maximum number of drops for f=4 : %d" % np.max(drops_f4))
print("Maximum number of drops for f=5 : %d" % np.max(drops_f5))

## Optimum Strategy
If we loop over all possible $f$ strategies and calculate the maximum number of drops for each case of $f$ we find that the optimum strategy is for $f=10$. We can mathematically prove that this is the case by imposing that the derivative of $D_{max}$ with respect to $f$ must be zero:

\begin{equation}
\frac{d D_{max}}{d f} = \frac{d }{d f}[ \frac{M}{f} + (f - 1)] = 1 - \frac{M}{f^2} = 0
\end{equation}

so that the optimum strategy is $f=\sqrt{M}$.
If we input that value into the formula for the maximum number of drops we find that in the worst case scenario, for a building with $M$ floors following a strategy of $f=\sqrt{M}$ would find the $N$th floor in 

\begin{equation}
D_{max} = 2\sqrt{M} - 1
\end{equation}

In this case, for 100 floors, we would need a maximum of 19 drops. An example of such worst case scenario is if $N=99$. If we drop the first box every 10 floors, it will break at the 10th drop for the 100th floor. Then, we would need to try floors 91th through 99th with the second box, for a total of 19 drops.

In [ ]:
strategies = np.arange(2, 51)
max_drops = []
for f in strategies:
    drops_f = [drops(N=n, f=f) for n in N_floors]
    max_drops.append(np.max(drops_f))
    
fig, ax = plt.subplots(1, 1)
ax.scatter(strategies, max_drops)
ax.set_xlabel(r'$f$ strategy')
ax.set_ylabel(r'')
plt.show()